In [1]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [2]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [3]:


gamma=0.7
epsilon_decay = 0.005
learning_rate =0.0005
max_e=1
min_e=0
replay_batch_size=30
iterations=2000

In [4]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
target_model =initiate_NN(learning_rate)
target_model.set_weights(online_model.get_weights())
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -108.89  rolling reward:  -108.89  Steps:  76  epsilon:  1.00
episode:  1  episode reward:  -110.20  rolling reward:  -109.55  Steps:  84  epsilon:  1.00
episode:  2  episode reward:  -219.16  rolling reward:  -146.09  Steps:  64  epsilon:  0.99
episode:  3  episode reward:  -398.50  rolling reward:  -209.19  Steps:  93  epsilon:  0.99
episode:  4  episode reward:  -77.15  rolling reward:  -182.78  Steps:  71  epsilon:  0.98
episode:  5  episode reward:  -209.01  rolling reward:  -187.15  Steps:  158  epsilon:  0.98
episode:  6  episode reward:  -399.73  rolling reward:  -217.52  Steps:  94  epsilon:  0.97
episode:  7  episode reward:  -277.11  rolling reward:  -224.97  Steps:  99  epsilon:  0.97
episode:  8  episode reward:  -112.93  rolling reward:  -212.52  Steps:  120  epsilon:  0.96
episode:  9  episode reward:  -105.91  rolling reward:  -201.86  Steps:  71  epsilon:  0.96
episode:  10  episode reward:  -335.12  rolling reward:  -213.97  Steps:  102  

episode:  89  episode reward:  -53.86  rolling reward:  -146.99  Steps:  114  epsilon:  0.64
episode:  90  episode reward:  -154.97  rolling reward:  -147.08  Steps:  158  epsilon:  0.64
episode:  91  episode reward:  -106.41  rolling reward:  -146.64  Steps:  87  epsilon:  0.63
episode:  92  episode reward:  -63.23  rolling reward:  -145.74  Steps:  116  epsilon:  0.63
episode:  93  episode reward:  -224.62  rolling reward:  -146.58  Steps:  122  epsilon:  0.63
episode:  94  episode reward:  -80.24  rolling reward:  -145.88  Steps:  150  epsilon:  0.63
episode:  95  episode reward:  -63.45  rolling reward:  -145.02  Steps:  98  epsilon:  0.62
episode:  96  episode reward:  -57.47  rolling reward:  -144.12  Steps:  119  epsilon:  0.62
episode:  97  episode reward:  -110.19  rolling reward:  -143.77  Steps:  278  epsilon:  0.62
episode:  98  episode reward:  -148.88  rolling reward:  -143.82  Steps:  121  epsilon:  0.61
episode:  99  episode reward:  -63.78  rolling reward:  -143.02  St

episode:  177  episode reward:  -153.93  rolling reward:  -103.10  Steps:  260  epsilon:  0.41
episode:  178  episode reward:  -232.93  rolling reward:  -104.33  Steps:  288  epsilon:  0.41
episode:  179  episode reward:  -173.96  rolling reward:  -105.22  Steps:  168  epsilon:  0.41
episode:  180  episode reward:  -108.36  rolling reward:  -104.08  Steps:  263  epsilon:  0.41
episode:  181  episode reward:  -108.73  rolling reward:  -103.63  Steps:  171  epsilon:  0.40
episode:  182  episode reward:  -232.86  rolling reward:  -104.48  Steps:  273  epsilon:  0.40
episode:  183  episode reward:  -76.57  rolling reward:  -103.64  Steps:  166  epsilon:  0.40
episode:  184  episode reward:  -208.94  rolling reward:  -104.49  Steps:  205  epsilon:  0.40
episode:  185  episode reward:  -236.89  rolling reward:  -105.96  Steps:  244  epsilon:  0.40
episode:  186  episode reward:  -170.10  rolling reward:  -106.49  Steps:  275  epsilon:  0.39
episode:  187  episode reward:  -96.33  rolling rew

KeyboardInterrupt: 

In [5]:

for episode in range(206,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  206  episode reward:  -80.47  rolling reward:  -115.47  Steps:  275  epsilon:  0.36
episode:  207  episode reward:  -238.76  rolling reward:  -116.94  Steps:  371  epsilon:  0.36
episode:  208  episode reward:  -201.64  rolling reward:  -117.69  Steps:  240  epsilon:  0.35
episode:  209  episode reward:  -87.44  rolling reward:  -117.84  Steps:  187  epsilon:  0.35
episode:  210  episode reward:  36.38  rolling reward:  -116.52  Steps:  192  epsilon:  0.35
episode:  211  episode reward:  -23.43  rolling reward:  -116.19  Steps:  314  epsilon:  0.35
episode:  212  episode reward:  -28.53  rolling reward:  -115.41  Steps:  201  epsilon:  0.35
episode:  213  episode reward:  9.81  rolling reward:  -114.42  Steps:  122  epsilon:  0.34
episode:  214  episode reward:  -45.60  rolling reward:  -112.77  Steps:  254  epsilon:  0.34
episode:  215  episode reward:  -62.92  rolling reward:  -111.52  Steps:  184  epsilon:  0.34
episode:  216  episode reward:  -10.32  rolling reward:  -111

KeyboardInterrupt: 

In [6]:

for episode in range(245,iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                target_q = target_model.predict(np.vstack(batch[:, 3]))
                y[non_terminal_samples] += np.multiply(gamma, \
                            target_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
        if done:
            target_model.set_weights(online_model.get_weights())

    episode_reward.append(current_episode_reward) 
    if len(memory)>=3000:
        memory = memory[-3000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  245  episode reward:  24.91  rolling reward:  -113.80  Steps:  1000  epsilon:  0.29
episode:  246  episode reward:  27.76  rolling reward:  -113.44  Steps:  239  epsilon:  0.29
episode:  247  episode reward:  2.86  rolling reward:  -112.71  Steps:  278  epsilon:  0.29
episode:  248  episode reward:  -39.14  rolling reward:  -112.03  Steps:  1000  epsilon:  0.29
episode:  249  episode reward:  -135.12  rolling reward:  -112.47  Steps:  65  epsilon:  0.29
episode:  250  episode reward:  -176.76  rolling reward:  -113.37  Steps:  311  epsilon:  0.29
episode:  251  episode reward:  -2.92  rolling reward:  -113.03  Steps:  317  epsilon:  0.29
episode:  252  episode reward:  67.78  rolling reward:  -111.95  Steps:  1000  epsilon:  0.28
episode:  253  episode reward:  73.09  rolling reward:  -108.15  Steps:  1000  epsilon:  0.28
episode:  254  episode reward:  -53.95  rolling reward:  -107.87  Steps:  214  epsilon:  0.28
episode:  255  episode reward:  -77.24  rolling reward:  -107.

episode:  333  episode reward:  -253.74  rolling reward:  -79.84  Steps:  181  epsilon:  0.19
episode:  334  episode reward:  -113.70  rolling reward:  -79.32  Steps:  233  epsilon:  0.19
episode:  335  episode reward:  -228.82  rolling reward:  -82.37  Steps:  389  epsilon:  0.19
episode:  336  episode reward:  -25.43  rolling reward:  -80.23  Steps:  294  epsilon:  0.19
episode:  337  episode reward:  -68.71  rolling reward:  -78.52  Steps:  189  epsilon:  0.19
episode:  338  episode reward:  -5.68  rolling reward:  -77.10  Steps:  276  epsilon:  0.18
episode:  339  episode reward:  -224.26  rolling reward:  -77.43  Steps:  440  epsilon:  0.18
episode:  340  episode reward:  12.60  rolling reward:  -76.49  Steps:  269  epsilon:  0.18
episode:  341  episode reward:  -128.58  rolling reward:  -76.46  Steps:  113  epsilon:  0.18
episode:  342  episode reward:  -193.17  rolling reward:  -78.25  Steps:  172  epsilon:  0.18
episode:  343  episode reward:  -68.38  rolling reward:  -76.94  S

episode:  421  episode reward:  -113.38  rolling reward:  -55.54  Steps:  105  epsilon:  0.12
episode:  422  episode reward:  -50.11  rolling reward:  -54.00  Steps:  181  epsilon:  0.12
episode:  423  episode reward:  123.05  rolling reward:  -52.06  Steps:  825  epsilon:  0.12
episode:  424  episode reward:  -168.39  rolling reward:  -52.88  Steps:  284  epsilon:  0.12
episode:  425  episode reward:  210.65  rolling reward:  -51.35  Steps:  687  epsilon:  0.12
episode:  426  episode reward:  -75.34  rolling reward:  -50.47  Steps:  295  epsilon:  0.12
episode:  427  episode reward:  -13.92  rolling reward:  -53.45  Steps:  351  epsilon:  0.12
episode:  428  episode reward:  -33.30  rolling reward:  -52.67  Steps:  456  epsilon:  0.12
episode:  429  episode reward:  -28.87  rolling reward:  -52.44  Steps:  401  epsilon:  0.12
episode:  430  episode reward:  -185.25  rolling reward:  -53.39  Steps:  318  epsilon:  0.12
episode:  431  episode reward:  -172.98  rolling reward:  -56.64  S

episode:  509  episode reward:  -250.50  rolling reward:  -48.19  Steps:  431  epsilon:  0.08
episode:  510  episode reward:  -247.88  rolling reward:  -49.27  Steps:  400  epsilon:  0.08
episode:  511  episode reward:  230.39  rolling reward:  -44.37  Steps:  519  epsilon:  0.08
episode:  512  episode reward:  -135.42  rolling reward:  -43.56  Steps:  1000  epsilon:  0.08
episode:  513  episode reward:  -206.57  rolling reward:  -45.78  Steps:  209  epsilon:  0.08
episode:  514  episode reward:  232.44  rolling reward:  -43.28  Steps:  387  epsilon:  0.08
episode:  515  episode reward:  265.76  rolling reward:  -40.72  Steps:  468  epsilon:  0.08
episode:  516  episode reward:  -11.81  rolling reward:  -40.52  Steps:  295  epsilon:  0.08
episode:  517  episode reward:  -139.02  rolling reward:  -41.04  Steps:  1000  epsilon:  0.08
episode:  518  episode reward:  -10.76  rolling reward:  -38.97  Steps:  212  epsilon:  0.08
episode:  519  episode reward:  195.36  rolling reward:  -39.40

episode:  597  episode reward:  -125.10  rolling reward:  -57.16  Steps:  1000  epsilon:  0.05
episode:  598  episode reward:  -47.34  rolling reward:  -60.19  Steps:  375  epsilon:  0.05
episode:  599  episode reward:  -77.77  rolling reward:  -60.80  Steps:  492  epsilon:  0.05
episode:  600  episode reward:  -192.40  rolling reward:  -61.08  Steps:  341  epsilon:  0.05
episode:  601  episode reward:  -177.41  rolling reward:  -62.65  Steps:  313  epsilon:  0.05
episode:  602  episode reward:  -232.52  rolling reward:  -62.57  Steps:  435  epsilon:  0.05
episode:  603  episode reward:  -78.74  rolling reward:  -65.99  Steps:  445  epsilon:  0.05
episode:  604  episode reward:  -116.79  rolling reward:  -65.43  Steps:  1000  epsilon:  0.05
episode:  605  episode reward:  -145.22  rolling reward:  -63.90  Steps:  568  epsilon:  0.05
episode:  606  episode reward:  -214.31  rolling reward:  -67.78  Steps:  295  epsilon:  0.05
episode:  607  episode reward:  -218.53  rolling reward:  -69

episode:  685  episode reward:  -332.96  rolling reward:  -79.36  Steps:  778  epsilon:  0.03
episode:  686  episode reward:  -29.29  rolling reward:  -81.81  Steps:  280  epsilon:  0.03
episode:  687  episode reward:  -302.93  rolling reward:  -86.67  Steps:  515  epsilon:  0.03
episode:  688  episode reward:  -266.01  rolling reward:  -86.87  Steps:  232  epsilon:  0.03
episode:  689  episode reward:  -303.16  rolling reward:  -89.82  Steps:  344  epsilon:  0.03
episode:  690  episode reward:  -133.06  rolling reward:  -90.59  Steps:  1000  epsilon:  0.03
episode:  691  episode reward:  -58.53  rolling reward:  -89.65  Steps:  402  epsilon:  0.03
episode:  692  episode reward:  -466.10  rolling reward:  -92.91  Steps:  312  epsilon:  0.03
episode:  693  episode reward:  -55.73  rolling reward:  -92.45  Steps:  356  epsilon:  0.03
episode:  694  episode reward:  -223.11  rolling reward:  -92.76  Steps:  1000  epsilon:  0.03
episode:  695  episode reward:  -84.77  rolling reward:  -90.

episode:  773  episode reward:  -96.37  rolling reward:  -70.06  Steps:  319  epsilon:  0.02
episode:  774  episode reward:  -450.51  rolling reward:  -73.12  Steps:  352  epsilon:  0.02
episode:  775  episode reward:  -229.13  rolling reward:  -77.13  Steps:  451  epsilon:  0.02
episode:  776  episode reward:  -13.27  rolling reward:  -77.19  Steps:  350  epsilon:  0.02
episode:  777  episode reward:  -359.40  rolling reward:  -79.21  Steps:  796  epsilon:  0.02
episode:  778  episode reward:  214.95  rolling reward:  -74.55  Steps:  426  epsilon:  0.02
episode:  779  episode reward:  -241.32  rolling reward:  -76.63  Steps:  398  epsilon:  0.02
episode:  780  episode reward:  -231.41  rolling reward:  -80.92  Steps:  425  epsilon:  0.02
episode:  781  episode reward:  -58.76  rolling reward:  -79.83  Steps:  189  epsilon:  0.02
episode:  782  episode reward:  -146.54  rolling reward:  -79.48  Steps:  262  epsilon:  0.02
episode:  783  episode reward:  -52.65  rolling reward:  -79.81 

episode:  861  episode reward:  224.05  rolling reward:  -32.11  Steps:  409  epsilon:  0.01
episode:  862  episode reward:  14.55  rolling reward:  -30.93  Steps:  374  epsilon:  0.01
episode:  863  episode reward:  -143.80  rolling reward:  -31.83  Steps:  58  epsilon:  0.01
episode:  864  episode reward:  -61.15  rolling reward:  -32.67  Steps:  415  epsilon:  0.01
episode:  865  episode reward:  201.15  rolling reward:  -29.51  Steps:  545  epsilon:  0.01
episode:  866  episode reward:  196.13  rolling reward:  -25.56  Steps:  609  epsilon:  0.01
episode:  867  episode reward:  -228.49  rolling reward:  -27.18  Steps:  807  epsilon:  0.01
episode:  868  episode reward:  200.84  rolling reward:  -26.69  Steps:  665  epsilon:  0.01
episode:  869  episode reward:  222.81  rolling reward:  -23.93  Steps:  437  epsilon:  0.01
episode:  870  episode reward:  -35.93  rolling reward:  -24.33  Steps:  363  epsilon:  0.01
episode:  871  episode reward:  188.28  rolling reward:  -20.95  Steps

episode:  950  episode reward:  188.85  rolling reward:  -10.92  Steps:  488  epsilon:  0.01
episode:  951  episode reward:  -3.30  rolling reward:  -10.23  Steps:  384  epsilon:  0.01
episode:  952  episode reward:  -204.82  rolling reward:  -14.13  Steps:  359  epsilon:  0.01
episode:  953  episode reward:  201.47  rolling reward:  -10.05  Steps:  714  epsilon:  0.01
episode:  954  episode reward:  -4.51  rolling reward:  -9.94  Steps:  246  epsilon:  0.01
episode:  955  episode reward:  -202.15  rolling reward:  -11.87  Steps:  290  epsilon:  0.01
episode:  956  episode reward:  166.66  rolling reward:  -9.85  Steps:  593  epsilon:  0.01
episode:  957  episode reward:  236.34  rolling reward:  -7.27  Steps:  423  epsilon:  0.01
episode:  958  episode reward:  -204.92  rolling reward:  -6.72  Steps:  426  epsilon:  0.01
episode:  959  episode reward:  201.29  rolling reward:  -6.58  Steps:  446  epsilon:  0.01
episode:  960  episode reward:  -191.68  rolling reward:  -10.88  Steps:  

KeyboardInterrupt: 

In [9]:
online_model.save_weights('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [8]:
len(episode_reward)

1004

In [11]:
'weights_DQN_'+str(max_e)+'-'+str(0.0001)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5'

'weights_DQN_1-0.0001-0.005-0.7-30-0.0005.h5'